In [4]:
import numpy as np
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import datetime
import sklearn

In [5]:
print(tf.config.list_physical_devices('GPU'))

[]


In [7]:
IMPORT_COUNT = 5000000
TEST_COUNT = 10000

In [8]:
# Generate random seed
#myrand=np.random.randint(1, 99999 + 1)
myrand=71926
np.random.seed(myrand)
tf.random.set_seed(myrand)
print("Random seed is:",myrand)

Random seed is: 71926


In [9]:
RNG_OUTPUT_FILENAME="mersenne_twist_states.txt"
df = np.genfromtxt(RNG_OUTPUT_FILENAME,delimiter='\n',dtype='uint64', max_rows=IMPORT_COUNT)

In [10]:
#calculates how many bits are in the output.
BIT_WIDTH=np.ceil(np.log2(np.amax(df))).astype(int)
# convert the generated numbers to binary sequences
df_as_bits =(df[:,None] & (1 << np.arange(BIT_WIDTH,dtype='uint64')) > 0).astype(int)

In [11]:
#convert the data into inputs and outputs
df_as_frames = np.array([df_as_bits[[X - 624, X - 624 + 1, X- 624 +397, X]] for X in range(624, df_as_bits.shape[0])])

In [12]:
# shuffle the data frames
indicies = np.arange(df_as_frames.shape[0],dtype='uint64')
np.random.shuffle(indicies)
df_as_frames=df_as_frames[indicies]

In [13]:
# convert the data into inputs and outputs
y = df_as_frames[:,-1,:]
X = df_as_frames[:,:-1,]
X = X.reshape([X.shape[0], X.shape[1]*X.shape[2]])[:, 31:]

In [14]:
# Split the data into train and test data
X_train = X[TEST_COUNT:]
X_test = X[:TEST_COUNT]
y_train = y[TEST_COUNT:]
y_test = y[:TEST_COUNT]

In [15]:
def build_model(hp):
    LOSS="binary_crossentropy"
    model = Sequential()
    model.add(Dense(96, activation='relu',input_shape=[X.shape[1]] ))
    model.add(Dense(y.shape[1], activation='sigmoid'))
    
    opt = keras.optimizers.Nadam(
        learning_rate=hp.Float("learning_rate", 10**(-5), 10**(-2),sampling="log"),
        epsilon=hp.Float("epsilon",1e-7,1e-4,sampling="log"),
        beta_1=hp.Float("beta_1",.8,.9999,sampling="reverse_log"),
        beta_2=hp.Float("beta_2",.8,.9999,sampling="reverse_log"),
        )
    model.compile(optimizer=opt, loss=LOSS,metrics=['binary_accuracy'])
    model.summary()
    return model

In [16]:
#define callback functions
stopEarly = tf.keras.callbacks.EarlyStopping(
    monitor='val_binary_accuracy', min_delta=.001, patience=10, verbose=0, mode='auto', restore_best_weights=False
)

log_dir = "tmp/hyperparameters_twisting/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,profile_batch=0)

In [17]:
# extract a short set from the training for hyper parameter tuning
X_train_short= X_train[:1000000]
y_train_short= y_train[:1000000]

In [18]:
%%time
# Tuning the hyper parameters

tuner = kt.tuners.bayesian.BayesianOptimization(build_model,'binary_accuracy',25,project_name="tmp/twisting", seed=myrand)

while tuner.remaining_trials>0:
    try:
        tuner.search(X_train_short, y_train_short,batch_size=2048, epochs=100, validation_data=(X_test,y_test),callbacks=[stopEarly,tensorboard_callback])
    except:
        pass

tuner.results_summary()

INFO:tensorflow:Reloading Tuner from .\tmp/twisting\tuner0.json
Results summary
Results in .\tmp/twisting
Showing 10 best trials
Trial summary
Hyperparameters:
learning_rate: 0.01
epsilon: 8.517143018543503e-05
beta_1: 0.8787368405865883
beta_2: 0.9486251178169223
Score: 1.0
Trial summary
Hyperparameters:
learning_rate: 0.01
epsilon: 8.781877460199428e-06
beta_1: 0.9159972966614436
beta_2: 0.9719095083753959
Score: 1.0
Trial summary
Hyperparameters:
learning_rate: 0.0035977846628660187
epsilon: 2.3823395017282265e-06
beta_1: 0.9398678587731861
beta_2: 0.9693012738278074
Score: 0.9836973547935486
Trial summary
Hyperparameters:
learning_rate: 0.002813155631740426
epsilon: 6.86915529734392e-06
beta_1: 0.933608723098096
beta_2: 0.9999
Score: 0.9688019752502441
Trial summary
Hyperparameters:
learning_rate: 0.01
epsilon: 0.0001
beta_1: 0.8045360648031332
beta_2: 0.9114469946180094
Score: 0.9687929749488831
Trial summary
Hyperparameters:
learning_rate: 0.0031481389496828293
epsilon: 3.7197322

In [19]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
# use the best model for training
model = tuner.hypermodel.build(best_hps)

best_hps.values

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 96)                6336      
                                                                 
 dense_1 (Dense)             (None, 32)                3104      
                                                                 
Total params: 9,440
Trainable params: 9,440
Non-trainable params: 0
_________________________________________________________________


{'learning_rate': 0.01,
 'epsilon': 8.517143018543503e-05,
 'beta_1': 0.8787368405865883,
 'beta_2': 0.9486251178169223}

In [20]:
def train_model(model, X = X_train, Y=y_train, epochs=100, batch_size=2048,verbose=1, log_dir = "tmp/dense_model/"):
    log_dir += datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,profile_batch=0)
    model.fit(X, Y, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size,callbacks=[tensorboard_callback],verbose=verbose)
    return model

In [21]:
%%time
model = train_model(model, epochs=20,log_dir = "tmp/mt_twisting_model/")

Epoch 1/20
2437/2437 [==============================] - 14s 4ms/step - loss: 0.2712 - binary_accuracy: 0.8180 - val_loss: 0.1852 - val_binary_accuracy: 0.8711
Epoch 2/20
2437/2437 [==============================] - 10s 4ms/step - loss: 0.1506 - binary_accuracy: 0.8951 - val_loss: 0.1149 - val_binary_accuracy: 0.9205
Epoch 3/20
2437/2437 [==============================] - 10s 4ms/step - loss: 0.1091 - binary_accuracy: 0.9219 - val_loss: 0.1095 - val_binary_accuracy: 0.9217
Epoch 4/20
2437/2437 [==============================] - 10s 4ms/step - loss: 0.0975 - binary_accuracy: 0.9316 - val_loss: 0.0662 - val_binary_accuracy: 0.9528
Epoch 5/20
2437/2437 [==============================] - 10s 4ms/step - loss: 0.0651 - binary_accuracy: 0.9540 - val_loss: 0.0636 - val_binary_accuracy: 0.9559
Epoch 6/20
2437/2437 [==============================] - 10s 4ms/step - loss: 0.0470 - binary_accuracy: 0.9675 - val_loss: 0.0440 - val_binary_accuracy: 0.9688
Epoch 7/20
2437/2437 [========================

In [18]:
results = model.evaluate(X_test, y_test, batch_size=256)
print("test loss: %f, test acc: %s" % tuple(results))

40/40 [==============================] - 0s 2ms/step - loss: 1.3283e-08 - binary_accuracy: 1.0000
test loss: 0.000000, test acc: 1.0


In [19]:
model.save("mt_twisting_model.h5")